In [427]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np
from JoyStick import Joystick
#from Character import Character


In [428]:
class Character:
  def __init__(self):
    # 캐릭터에 사용할 이미지
    self.cat_front = Image.open('images/character/cat_front.png').convert('RGBA')
    self.cat_left = Image.open('images/character/cat_left.png').convert('RGBA')
    self.cat_right = Image.open('images/character/cat_right.png').convert('RGBA')
    self.cat_angry = Image.open('images/character/cat_angry.png').convert('RGBA')
    # 이미지 크기 줄이기 
    self.cat_front_resized = self.cat_front.resize((70, 70)) 
    self.cat_left_resized = self.cat_left.resize((70, 70)) 
    self.cat_right_resized = self.cat_right.resize((70, 70)) 
    self.cat_angry_resized = self.cat_angry.resize((70, 70)) 
    # 캐릭터 위치
    self.x_position = 120
    self.y_position = 180
    #self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 +20])
    self.speed = 5
  # 위 아래로는 움직이지 않음
  def move(self,up_down,left_right):
      if up_down=='up_pressed':
           pass
      elif up_down == 'down_prssed':
           pass
      elif left_right == 'left_pressed':
          self.x_position -= self.speed
      elif left_right == 'right_pressed':
          self.x_position += self.speed


In [429]:
class Fish:
  def __init__(self,x1,y1,x2,y2):
    self.image = Image.open('images/character/fish.png').convert('RGBA')
    self.fish_resized = self.image.resize((50, 50)) 
    self.left_top_x = x1
    self.left_top_y = y1
    self.right_bottom_x = x2
    self.right_bottom_y = y2
    self.speed = 5
  def randomMove(self):
        #y만 self.speed 만큼 + 하면서 내려오게 하면 됨
        if self.left_top_y < 240 :
            self.left_top_y = self.left_top_y + self.speed
            self.right_bottom_y = self.right_bottom_y + self.Enemy_speed

  def hit(self, cat):
        if((cat.position[0] > self.left_top_x and cat.position[2] < self.right_bottom_x ) and \
            (cat.position[1] < self.right_bottom_y and cat.position[3] > self.left_top_y)) or \
            ((cat.position[0] + 25> self.left_top_x and cat.position[2] + 25 < self.right_bottom_x) and \
                (cat.position[1] < self.right_bottom_y and cat.position[3] > self.left_top_y)):
            print("collision!!!")
            #뒤로 갈수록 점점 속도가 빨라져 앞에 부딪히는 것만 인식하기 때문에, 각 변의 길이와 좌표를 이용하여 부딪히는지 안부딪히는지 체크함 직사각형으로 따지면 밑의 부분
            return 1

        else:
            return 0

In [430]:
class Fishes:
    def __init__(self):
      self.fish_list = []
      self.x1 = 0
      self.x2 = 25
      self.y1 = 0
      self.y2 = 25
      for i in range(0, 8):
          self.fish_list.append(Fish(self.x1, self.y1, self.x2, self.y2))
          self.x1 += 30
          self.x2 += 30

In [431]:
class Background:
  def __init__(self):
    self.backgroundList=[]
    name="background"
    for i in range(10) :
      background = Image.open('images/background/'+name+str(i+1)+'.png').convert('RGBA')
      self.backgroundList.append(background)

In [432]:
# joystick 객체
joystick = Joystick()

# 캐릭터 객체 생성
cat = Character()
# fish 객체 생성
fishes = Fishes()


# Get drawing object to draw on image
image = Image.new("RGBA", (joystick.width, joystick.height))
draw =ImageDraw.Draw(image)
# clear display
draw.rectangle((0, 0, joystick.width, joystick.height), outline=0, fill=(255, 0, 0))
joystick.disp.image(image)

In [433]:
#background = Image.open('images/background/background1.png')
#background = background.resize((240,240))
# for i in range(10):
#   background = Background().backgroundList[i]
#   background = background.resize((240,240))
background = Background()
total_backgrounds=len(background.backgroundList)
current_background_index = 0

left_right = 'None'


In [434]:


while True:
    time.sleep(0.01)
    current_background_index = (current_background_index + 1) % total_backgrounds
    current_background = background.backgroundList[current_background_index]
    current_background = current_background.resize((240, 240))
    image.paste(current_background, (0, 0))

    up_down = 'None'
    left_right = 'None'
    if not joystick.button_U.value:
         up_down = 'up_pressed'
    if not joystick.button_D.value:
         up_down = 'down_pressed'
    if not joystick.button_L.value:
        left_right = 'left_pressed'
    if not joystick.button_R.value:
        left_right = 'right_pressed'

    # 움직이게 만들기 
    cat.move(up_down, left_right)
    # 가만히 있을 때 이미지표시
    image.paste(cat.cat_front_resized, (cat.x_position,cat.y_position))
    # 왼쪽 오른쪽으로 움직일 때 다른 이미지로 보이도록 함.
    if(left_right =='left_pressed') :
        image.paste(cat.cat_left_resized, (cat.x_position,cat.y_position))
    elif(left_right == 'right_pressed'):
         image.paste(cat.cat_right_resized, (cat.x_position,cat.y_position))
    
    
    joystick.disp.image(image)


KeyboardInterrupt: 